In [ ]:
!pip install ktrain

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from ktrain import text
import pandas as pd
import numpy as np
import ktrain
import nltk
import re

In [3]:
train = pd.read_csv("https://raw.githubusercontent.com/ishantjuyal/Emotions-Detection/main/Data/Emotion_final.csv")

In [4]:
print("Shape of train is", train.shape)
train.head()

Shape of train is (21459, 2)


,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [5]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df= train, 
                                                                      text_column = 'Text',
                                                                      label_columns = 'Emotion',
                                                                      maxlen = 50,
                                                                      preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [6]:
model = text.text_classifier(name= 'bert',
                             train_data= (X_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 50
done.


In [7]:
# Get Learning Rate
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 32)

In [8]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
604/604 [==============================] - 334s 552ms/step - loss: 0.8173 - accuracy: 0.6921 - val_loss: 0.2492 - val_accuracy: 0.8984


In [9]:
predictor = ktrain.get_predictor(learner.model, preproc= preprocess)

In [10]:
data = ["I have really become used to this cycle where I stay happy for 14-15 days and then spend the next 5-6 days feeling worthless and feeling like doing nothing at all.These emotions are not clear, they are not the primary emotions that I face because I don’t have a clear picture of what I am feeling.I haven’t felt love, I have stopped feeling any kind of emotion and I have become numb, completely numb.This feeling of emptiness, like a void which cannot be filled now.Even when the lockdown is not there, I feel like I am in lockdown.I guess this 3 weeks cycle is my routine now until things get better and life gets back to what it was."]

In [11]:
predictor.predict(data)

['sadness']

In [12]:
data = ["I am pretty sad and I don't have anyone to talk to.", 
        "I am great actually. Very happy with my life for now",
        "I am really scared of what is going to happen to me now"]

In [13]:
predictor.predict(data)

['sadness', 'happy', 'fear']

In [14]:
def detect(sentence):
    data = [sentence]
    emotion = predictor.predict(data)
    print('You are feeling ' + emotion[0])

In [15]:
detect("I have really become used to this cycle where I stay happy for 14-15 days and then spend the next 5-6 days feeling worthless and feeling like doing nothing at all.These emotions are not clear, they are not the primary emotions that I face because I don’t have a clear picture of what I am feeling.I haven’t felt love, I have stopped feeling any kind of emotion and I have become numb, completely numb.This feeling of emptiness, like a void which cannot be filled now.Even when the lockdown is not there, I feel like I am in lockdown.I guess this 3 weeks cycle is my routine now until things get better and life gets back to what it was.")

You are feeling sadness


In [17]:
data = np.array(train['Text'])
y_t = np.array(train['Emotion'])

In [ ]:
y_pred = predictor.predict(data)
y_pred = np.array(y_pred)

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
print("Accuracy score for test data is:")
print(accuracy_score(y_t, y_pred))
print("\nConfusion Matrix for test data is:")
print(confusion_matrix(y_t, y_pred))

Accuracy score for test data is:
0.9223635770539168

Confusion Matrix for test data is:
[[2765   85   21    8  113    1]
 [  71 2301   23    4   52  201]
 [  30   26 6479  364   75   55]
 [   5    4  162 1447   22    1]
 [  82   99   61   14 6006    3]
 [  16   44   20    0    4  795]]
